In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
sns.set_style("darkgrid")

In [ ]:
# Initial input paths were based on Kaggle's kernel notebook. 
train = pd.read_csv('../input/techjam2019/train.csv')
demograph = pd.read_csv('../input/techjam2019/demographics.csv')
kplus = pd.read_csv('../input/techjam2019/kplus.csv')
cc = pd.read_csv('../input/techjam2019/cc.csv')
test = pd.read_csv('../input/techjam2019/test.csv')


In [ ]:
demograph['ocp_cd'].fillna(demograph['ocp_cd'].mode().iloc[0], inplace=True)
demograph = demograph.astype({'ocp_cd': 'uint8',
                            'gender':'uint8',
                            'age':'uint8'})

In [ ]:
kplus['sunday'] = pd.to_datetime(kplus['sunday'])

In [ ]:
df_kp = kplus.groupby("id")[["kp_txn_count","kp_txn_amt"]].agg(["mean","sum"]).reset_index()
df_kp.columns = ["id", "kp_avg_count", "kp_sum_count", "kp_avg_amt", "kp_sum_amt"]

In [ ]:
df_kp["kp_totalamt_per_count"] = df_kp["kp_sum_amt"] / df_kp["kp_sum_count"]

In [ ]:
kplus['week'] = kplus['sunday'].dt.week
kplus['month'] = kplus['sunday'].dt.month
kplus['week'] = kplus['sunday'].dt.week

In [ ]:
kplus['kp_month_max_spending'] = kplus.groupby(['id','month'])['kp_txn_amt'].transform(max)
kplus['kp_month_min_spending'] = kplus.groupby(['id','month'])['kp_txn_amt'].transform(min)
kplus['kp_month_max_count'] = kplus.groupby(['id','month'])['kp_txn_count'].transform(max)
kplus['kp_month_min_count'] = kplus.groupby(['id','month'])['kp_txn_count'].transform(min)

k = kplus[kplus.groupby(['id','month'])['week'].transform(max) == kplus['week']]
col = ['kp_month_max_spending_max','kp_month_max_spending_mean',
       'kp_month_min_spending_min','kp_month_min_spending_mean',
       'kp_txn_amt_max_fin_week','kp_txn_amt_mean_fin_week',
       'kp_txn_amt_min_fin_week',
       'kp_txn_count_max_fin_week','kp_txn_count_mean_fin_week',
       'kp_txn_count_min_fin_week']

df_kp[col] = k.groupby(['id']).agg({'kp_month_max_spending': ['max','mean'],
                       'kp_month_min_spending': ['min','mean'],
                       'kp_txn_amt': ['max','mean','min'],
                       'kp_txn_count': ['max','mean','min']}).reset_index().drop('id', axis=1)


In [ ]:
k = kplus.groupby(['id','month'])['kp_txn_amt'].sum().reset_index()
# Try creating features based on monthly spending
for month in range(1,7):
    l = k[k.month == month].reset_index(drop=True).drop('month',axis=1)
    l.columns = ['id', 'kp_txn_amt_month_'+str(month)]
    df_kp = df_kp.merge(l, how="left", on="id")

In [ ]:
df_kp['kp_week_avg_interval'] = kplus.groupby('id')['week'].agg(lambda group: group.sort_values().diff().mean()).reset_index()['week']


In [ ]:
uniq_cc = cc["cc_no"].unique()
print("There are " + str(len(uniq_cc)) + " unique credit cards" )
print("There are " + str(len(cc)) + " transactions" )


In [ ]:
new_cc = cc.drop(["pos_dt"],axis=1).groupby("cc_no").agg(["sum","mean","count"]).reset_index()
new_cc.columns = ['cc_no','cc_sum','cc_mean','cc_count']
new_cc.head()

In [ ]:
cc['pos_dt'] = pd.to_datetime(cc['pos_dt'])
new_cc['cc_days_avg_interval'] = cc.groupby('cc_no')['pos_dt'].agg(lambda group: group.sort_values().diff().mean()).reset_index()['pos_dt']
new_cc['cc_days_avg_interval'] = new_cc['cc_days_avg_interval'].dt.days
new_cc['cc_days_avg_interval'].fillna(0, inplace=True)

In [ ]:
# Get the week and month
cc['month'] = cc.pos_dt.dt.month
cc['week'] = cc.pos_dt.dt.week

In [ ]:
# Get the max and min of number of trans
k = cc.groupby(['cc_no','month']).count()
new_cc['cc_month_trans_max'] = k.groupby('cc_no')['cc_txn_amt'].max().reset_index().drop('cc_no',axis=1)
new_cc['cc_month_trans_min'] = k.groupby('cc_no')['cc_txn_amt'].min().reset_index().drop('cc_no',axis=1)
new_cc['cc_month_trans_mean'] = k.groupby('cc_no')['cc_txn_amt'].mean().reset_index().drop('cc_no',axis=1)

In [ ]:
# Get the max, min, and mean of monthly spending
k = cc.groupby(['cc_no','month'])['cc_txn_amt'].sum().reset_index()
new_cc['cc_month_max_spend'] = k.groupby(['cc_no'])['cc_txn_amt'].max().reset_index().drop('cc_no',axis=1)
new_cc['cc_month_min_spend'] = k.groupby(['cc_no'])['cc_txn_amt'].min().reset_index().drop('cc_no',axis=1)
new_cc['cc_month_avg_spend'] = k.groupby(['cc_no'])['cc_txn_amt'].mean().reset_index().drop('cc_no',axis=1)

In [ ]:

for month in range(1,7):
    l = k[k.month == month].reset_index(drop=True).drop('month',axis=1)
    l.columns = ['cc_no', 'cc_txn_amt_month_'+str(month)]
    new_cc = new_cc.merge(l, how="left", on="cc_no")
new_cc.fillna(0, inplace=True)

In [ ]:
df_comb = pd.merge(demograph, new_cc, on="cc_no", how="left")
df_comb.fillna(0, inplace=True)
df_comb['cc_unuse'] = (df_comb.cc_count == 0).astype('int8')
df_comb.head()

In [ ]:
month_col = ["cc_txn_amt_month_"+str(month) for month in range(1,7)]

col = ["id", "cc_total_sum", "cc_total_mean", "cc_total_count","cc_days_avg_interval",
               "cc_month_trans_max","cc_month_trans_min","cc_month_trans_mean","cc_month_max_spend",
               "cc_month_min_spend", "cc_month_avg_spend","cc_unuse"] + month_col

tmp = df_comb.groupby("id")[["cc_sum", "cc_mean", "cc_count","cc_days_avg_interval", "cc_month_trans_max","cc_month_trans_min","cc_month_trans_mean",
                             "cc_month_max_spend",	"cc_month_min_spend", "cc_month_avg_spend", "cc_unuse"]+month_col].sum().reset_index()
tmp.columns = col

tmp['cc_num'] = df_comb.groupby('id').size().reset_index().iloc[:,1]
tmp.head()

In [ ]:
tmp = pd.merge(demograph.drop('cc_no', axis=1), tmp).drop_duplicates().reset_index(drop=True)

In [ ]:
tmp['cc_num_diff'] = tmp['cc_num'] - tmp['cc_unuse']
tmp["cc_total_mean"] = tmp['cc_total_mean'] / tmp['cc_num_diff']
tmp["cc_avg_total_sum"] = tmp['cc_total_sum'] / tmp['cc_num_diff']
tmp['cc_days_avg_interval'] = tmp['cc_days_avg_interval']  / tmp['cc_num_diff']

div = ["cc_total_mean","cc_days_avg_interval",
               "cc_month_trans_max","cc_month_trans_min","cc_month_trans_mean","cc_month_max_spend",
               "cc_month_min_spend", "cc_month_avg_spend"]


for i in div:
    tmp[i] = tmp[i] / tmp['cc_num_diff']
tmp.fillna(0, inplace=True)

tmp.head()

In [ ]:
tmp = pd.merge(tmp, df_kp, on="id", how="left")
tmp.fillna(0, inplace=True)
tmp.head()

In [ ]:
# Sum and diff
tmp["comb_spend"] = tmp["cc_total_sum"] + tmp["kp_sum_amt"]
tmp["comb_trans"] = tmp["cc_total_count"] + tmp["kp_sum_count"]
tmp["comb_spend_trans"] = tmp['comb_spend'] / tmp['comb_trans']
# Ratio
tmp["comb_spend_diff_ratio"] = np.abs(tmp['kp_sum_amt'] - tmp['cc_total_sum']) / tmp["comb_spend"]
tmp["comb_trans_diff_ratio"] = np.abs(tmp['kp_sum_count'] - tmp['cc_total_count']) / tmp["comb_trans"]
tmp["comb_kp_spend_ratio"] = tmp['kp_sum_amt'] / tmp['comb_spend']

tmp['cc_month_max_min_spend_rat'] = tmp['cc_month_max_spend']/tmp['cc_month_min_spend']
tmp['kp_month_max_min_spend_rat'] = tmp['kp_month_max_spending_max']/tmp['kp_month_min_spending_min']
tmp['kp_month_max_min_spend_mean_rat'] = tmp['kp_month_max_spending_mean'] / tmp['kp_month_min_spending_mean']

tmp['kp_txn_amt_max_min_finweek_rat'] = tmp['kp_txn_amt_max_fin_week'] / tmp['kp_txn_amt_min_fin_week']
tmp['kp_txn_count_max_min_finweek_rat'] =tmp['kp_txn_count_max_fin_week'] / tmp['kp_txn_count_min_fin_week']

tmp['cc_month_trans_max_min'] = tmp['cc_month_trans_max'] / tmp['cc_month_trans_min']
tmp["comb_kp_trans_ratio"] = tmp['kp_sum_count'] / tmp['comb_trans']


# Bad Feature
tmp.drop(['cc_num_diff'],axis=1, inplace=True)

In [ ]:
# Drop unimportant feature from feature selection from SHAP importance
bad_feat = ["comb_trans_diff_ratio","kp_txn_count_min_fin_week",
            "kp_txn_count_max_min_finweek_rat",
            "kp_txn_count_max_fin_week","kp_txn_amt_max_min_finweek_rat"]

tmp.drop(bad_feat,axis=1, inplace=True)

In [ ]:
train_df = pd.merge(train, tmp, on="id", how="left")
train_df.fillna(0, inplace=True)
train_df.head()

In [ ]:
def smape(F, A):
    return 100- ((100/len(A)) * np.sum(np.power(np.abs(F - A), 2) / np.power(np.minimum(2*np.abs(A), np.abs(F)) + np.abs(A), 2)))

def xgb_smape(F, A):
    A_label = A.get_label()
    return 'SMAPE',smape(np.exp(F), np.exp(A_label))


In [ ]:
# All non-zero income value, can use log instead of log1p.
train_df['income'] = np.log(train_df['income'])
sns.distplot(train_df['income'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(["income","id"],axis=1), train_df["income"], test_size = 0.3, random_state=42)

In [ ]:
# Use best hyperparameters to create a model
# XGB Regressor, GBT from hyperopt Bayesian Optimization
# 5-Fold CV Smape: 92.08001
# Validation Smape: 92.20557
params = {'colsample_bytree': 0.4074158302407245, 
 'gamma': 0.3630782973578546, 
 'learning_rate': 0.01001134673166224, 
 'max_depth': 9, 'min_child_weight': 7.0, 
 'random_state': 42, 
 'reg_alpha': 0.6057473986184368, 
 'reg_lambda': 0.9871588260368804, 
 'seed': 42,
 'nthread': -1, 
 'silent': 1, 
 'n_estimators': 434}

model =  xgb.XGBRegressor(**params)
model.fit(X_train, y_train)

In [ ]:
preds = np.exp(model.predict(X_test))
print('SMAPE from on test data = {:.5f}.'.format(smape(preds, np.exp(y_test))))

In [ ]:
# Implement feature engineering for test data here
test_df = pd.merge(test, tmp, how="left")
test_df.fillna(0, inplace=True)

# Predict the test data
test_preds = np.exp(model.predict(test_df.drop(['id'], axis=1)))
test['income'] = np.round(test_preds, 2)
test.to_csv('O_1033.csv', index = False, header=True)